In [ ]:
!pip install --upgrade pip
!pip install --ignore-installed --upgrade tensorflow==2.0.0-rc1

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import os
import time
import tarfile
import numpy as np

import sagemaker
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.predictor import json_serializer, json_deserializer

# Data

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
os.makedirs('data/train')
os.makedirs('data/test')

np.save('data/train/x.npy', x_train)
np.save('data/train/y.npy', y_train)
np.save('data/test/x.npy', x_test)
np.save('data/test/y.npy', y_test)

# Local

In [ ]:
%%sh

cd tensorflow-2-container/

chmod +x code/train
chmod +x code/serve

docker build  -t tensorflow-2 .

In [ ]:
job_name = ('tensorflow-2-' + time.strftime("%Y-%m-%d-%H-%M-%S")).replace('_', '-')

estimator = Estimator(image_name='tensorflow-2',
                      train_instance_type='local',
                      train_instance_count=1,
                      hyperparameters={'epochs': 1},
                      output_path='file:///home/ec2-user/SageMaker/sagemaker/tensorflow-2/output',
                      role=get_execution_role(),
                      base_job_name=job_name)

estimator.fit({'train': 'file:///home/ec2-user/SageMaker/sagemaker/tensorflow-2/data/train/', 
               'test': 'file:///home/ec2-user/SageMaker/sagemaker/tensorflow-2/data/test/'}, 
              job_name=job_name)

In [ ]:
predictor = estimator.deploy(1, 'local')

In [ ]:
data = {'instances': x_test[:2].tolist()}

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

predictor.predict(data)

In [ ]:
predictor.delete_endpoint()

In [ ]:
with tarfile.open('output/model.tar.gz', "r:gz") as tar:
    tar.extractall('output/model/')

In [ ]:
with tarfile.open('output/output.tar.gz', "r:gz") as tar:
    tar.extractall('output/output/')

# Remote

In [ ]:
bucket = 'gl-ml-training-sagemaker'
prefix = 'sagemaker/tensorflow-2'

In [ ]:
# Upload data
sagemaker.Session().upload_data(path='data/train', 
                                key_prefix=os.path.join(prefix, 'data', 'train'), 
                                bucket=bucket)
sagemaker.Session().upload_data(path='data/test', 
                                key_prefix=os.path.join(prefix, 'data', 'test'), 
                                bucket=bucket)

In [ ]:
# Build and upload container
%%sh

cd tensorflow-2-container/

image=tensorflow-2

chmod +x code/train
chmod +x code/serve

# Get the account number associated with the current IAM credentials
account=$(aws sts get-caller-identity --query Account --output text)

if [ $? -ne 0 ]
then
    exit 255
fi


# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}


fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${image}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${image} .
docker tag ${image} ${fullname}

docker push ${fullname}

In [ ]:
job_name = ('tensorflow-2-' + time.strftime("%Y-%m-%d-%H-%M-%S")).replace('_', '-')

estimator = Estimator(image_name='763331026866.dkr.ecr.us-east-1.amazonaws.com/tensorflow-2:latest',
                      train_instance_type='ml.m4.xlarge',
                      train_instance_count=1,
                      hyperparameters={'epochs': 1},
                      output_path='s3://{}/saegmaker/tensorflow-2/output'.format(bucket),
                      role=get_execution_role(),
                      base_job_name=job_name)

estimator.fit({'train': sagemaker.s3_input(s3_data='s3://{}/{}/data/train/'.format(bucket, prefix), content_type='csv'), 
               'test': sagemaker.s3_input(s3_data='s3://{}/{}/data/test/'.format(bucket, prefix), content_type='csv')}, 
              job_name=job_name)

In [ ]:
predictor = estimator.deploy(1, 'ml.m4.xlarge')

In [ ]:
data = {'instances': x_test.tolist()}

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

predictor.predict(data)